##### Case Study - Whats Cooking (Kaggle Dataset)

Import Libararies 

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
import re
import nltk
import random

from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore")

Read Datafiles 

In [2]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')
submit = pd.read_csv("sample_submission.csv")

#### Exploratory Data Analysis 

In [3]:
train.head(2)
test.head(2)
submit.head(2)

,id,cuisine
0,35203,italian
1,17600,italian


In [4]:
# Removing space within words from the ingredients column 
new = []
for items in train.ingredients:
    items = [item.replace(' ', '') for item in items]
    new.append(items)

In [5]:
new_test = []
for items in test.ingredients:
    items = [item.replace(' ', '') for item in items]
    new_test.append(items)

In [6]:
# Removing List of Lists and creating additional column with new ingredient list 
print(new[0:10])
new_transposed = zip(new)
new = pd.DataFrame(new_transposed)
print(new.head())

[['romainelettuce', 'blackolives', 'grapetomatoes', 'garlic', 'pepper', 'purpleonion', 'seasoning', 'garbanzobeans', 'fetacheesecrumbles'], ['plainflour', 'groundpepper', 'salt', 'tomatoes', 'groundblackpepper', 'thyme', 'eggs', 'greentomatoes', 'yellowcornmeal', 'milk', 'vegetableoil'], ['eggs', 'pepper', 'salt', 'mayonaise', 'cookingoil', 'greenchilies', 'grilledchickenbreasts', 'garlicpowder', 'yellowonion', 'soysauce', 'butter', 'chickenlivers'], ['water', 'vegetableoil', 'wheat', 'salt'], ['blackpepper', 'shallots', 'cornflour', 'cayennepepper', 'onions', 'garlicpaste', 'milk', 'butter', 'salt', 'lemonjuice', 'water', 'chilipowder', 'passata', 'oil', 'groundcumin', 'bonelesschickenskinlessthigh', 'garammasala', 'doublecream', 'naturalyogurt', 'bayleaf'], ['plainflour', 'sugar', 'butter', 'eggs', 'freshgingerroot', 'salt', 'groundcinnamon', 'milk', 'vanillaextract', 'groundginger', 'powderedsugar', 'bakingpowder'], ['oliveoil', 'salt', 'mediumshrimp', 'pepper', 'garlic', 'choppedci

In [7]:
print(new_test[0:10])
new_test_transposed = zip(new_test)
new_test = pd.DataFrame(new_test_transposed)
print(new_test.head())

[['bakingpowder', 'eggs', 'all-purposeflour', 'raisins', 'milk', 'whitesugar'], ['sugar', 'eggyolks', 'cornstarch', 'creamoftartar', 'bananas', 'vanillawafers', 'milk', 'vanillaextract', 'toastedpecans', 'eggwhites', 'lightrum'], ['sausagelinks', 'fennelbulb', 'fronds', 'oliveoil', 'cubanpeppers', 'onions'], ['meatcuts', 'filepowder', 'smokedsausage', 'okra', 'shrimp', 'andouillesausage', 'water', 'paprika', 'hotsauce', 'garliccloves', 'browning', 'lumpcrabmeat', 'vegetableoil', 'all-purposeflour', 'freshlygroundpepper', 'flatleafparsley', 'bonelesschickenskinlessthigh', 'driedthyme', 'whiterice', 'yellowonion', 'ham'], ['groundblackpepper', 'salt', 'sausagecasings', 'leeks', 'parmigianoreggianocheese', 'cornmeal', 'water', 'extra-virginoliveoil'], ['bakingpowder', 'all-purposeflour', 'peachslices', 'cornstarch', 'heavycream', 'lemonjuice', 'unsaltedbutter', 'salt', 'whitesugar'], ['grapejuice', 'orange', 'whitezinfandel'], ['groundginger', 'whitepepper', 'greenonions', 'orangejuice', 

In [8]:
# conacate additional column with train
train = pd.concat([train,new],axis = 1)

In [9]:
test = pd.concat([test,new_test],axis = 1)

In [10]:
# Renaming new column 
train = train.rename(columns={0: "ingredients_2"})

In [11]:
test = test.rename(columns={0: "ingredients_2"})

In [12]:
train.head(2)

,id,cuisine,ingredients,ingredients_2
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...","[romainelettuce, blackolives, grapetomatoes, g..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","[plainflour, groundpepper, salt, tomatoes, gro..."


In [13]:
test.head(2)

,id,ingredients,ingredients_2
0,18009,"[baking powder, eggs, all-purpose flour, raisi...","[bakingpowder, eggs, all-purposeflour, raisins..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta...","[sugar, eggyolks, cornstarch, creamoftartar, b..."


In [14]:
# Removing commas (',' ) from ingredient2 and saving in data in additional column 
train['ingredients_1'] =train['ingredients_2'].apply(' '.join)

In [15]:
test['ingredients_1'] =test['ingredients_2'].apply(' '.join)

In [16]:
train.head()

,id,cuisine,ingredients,ingredients_2,ingredients_1
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...","[romainelettuce, blackolives, grapetomatoes, g...",romainelettuce blackolives grapetomatoes garli...
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","[plainflour, groundpepper, salt, tomatoes, gro...",plainflour groundpepper salt tomatoes groundbl...
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...","[eggs, pepper, salt, mayonaise, cookingoil, gr...",eggs pepper salt mayonaise cookingoil greenchi...
3,22213,indian,"[water, vegetable oil, wheat, salt]","[water, vegetableoil, wheat, salt]",water vegetableoil wheat salt
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...","[blackpepper, shallots, cornflour, cayennepepp...",blackpepper shallots cornflour cayennepepper o...


In [17]:
test.head(2)

,id,ingredients,ingredients_2,ingredients_1
0,18009,"[baking powder, eggs, all-purpose flour, raisi...","[bakingpowder, eggs, all-purposeflour, raisins...",bakingpowder eggs all-purposeflour raisins mil...
1,28583,"[sugar, egg yolks, corn starch, cream of tarta...","[sugar, eggyolks, cornstarch, creamoftartar, b...",sugar eggyolks cornstarch creamoftartar banana...


In [18]:
# Train-Test Split 
train_corpus = train['ingredients_1']
test_corpus = test['ingredients_1']
y = train['cuisine']
X  = train_corpus

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [19]:
print(train_corpus.shape)
print(test_corpus.shape)
print(X_train.shape)
print(X_test.shape)

(39774,)
(9944,)
(27841,)
(11933,)


In [20]:
# Remove array
#train['ingredients_clean_string'] = [' , '.join(z).strip() for z in train['ingredients']]   
#test['ingredients_clean_string'] = [' , '.join(z).strip() for z in test['ingredients']]

#### Creating TFIDF vector metrics 

In [21]:
train_vectorizer = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ),analyzer="word", 
                             max_df = .57 , binary=False , token_pattern=r'\w+' , sublinear_tf=False)

In [22]:
train_tfidf=train_vectorizer.fit_transform(X_train).todense()

In [23]:
train_tfidf.shape

(27841, 6171)

In [24]:
val_tfidf = train_vectorizer.transform(X_test)

In [25]:
val_tfidf.shape

(11933, 6171)

In [26]:
test_tfidf=train_vectorizer.transform(test_corpus)

In [27]:
test_tfidf.shape

(9944, 6171)

In [28]:
train_tfidf

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
val_tfidf

<11933x6171 sparse matrix of type '<class 'numpy.float64'>'
	with 130090 stored elements in Compressed Sparse Row format>

In [30]:
test_tfidf

<9944x6171 sparse matrix of type '<class 'numpy.float64'>'
	with 108646 stored elements in Compressed Sparse Row format>

#### Model - 1 SVC

In [31]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
model = LinearSVC(C=0.80, penalty="l2", dual=False)
parameters = {'C':[1, 10]}
classifier = GridSearchCV(model, parameters)
classifier.fit(train_tfidf,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=LinearSVC(C=0.8, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=None, param_grid={'C': [1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [32]:
train_pred = classifier.predict(train_tfidf)

In [33]:
val_pred = classifier.predict(val_tfidf)
#test_predict = classifier.predict(test)

In [34]:
test_pred = classifier.predict(test_tfidf)

In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_train, train_pred))
print(classification_report(y_test, val_pred))

              precision    recall  f1-score   support

   brazilian       0.95      0.86      0.90       333
     british       0.87      0.76      0.81       582
cajun_creole       0.90      0.87      0.88      1114
     chinese       0.90      0.95      0.93      1870
    filipino       0.94      0.88      0.91       534
      french       0.83      0.82      0.82      1857
       greek       0.93      0.86      0.90       819
      indian       0.93      0.97      0.95      2094
       irish       0.92      0.80      0.86       483
     italian       0.90      0.96      0.93      5432
    jamaican       0.95      0.91      0.93       375
    japanese       0.95      0.85      0.90       994
      korean       0.96      0.93      0.95       576
     mexican       0.96      0.97      0.96      4482
    moroccan       0.95      0.93      0.94       561
     russian       0.90      0.79      0.84       345
 southern_us       0.86      0.92      0.89      3065
     spanish       0.88    

In [36]:
test_predict = pd.DataFrame(test_pred)
sub = pd.concat([test['id'],test_predict],axis=1)

In [37]:
sub.head()

,id,0
0,18009,british
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian


In [38]:
sub.to_csv("svc.csv",index=True)

In [39]:
submission = pd.read_csv('sample_submission.csv')

In [40]:
submission.head(5)

,id,cuisine
0,35203,italian
1,17600,italian
2,35200,italian
3,17602,italian
4,17605,italian


In [41]:
test['id']

0       18009
1       28583
2       41580
3       29752
4       35687
        ...  
9939    30246
9940    36028
9941    22339
9942    42525
9943     1443
Name: id, Length: 9944, dtype: int64

In [42]:
test_pred = pd.DataFrame(test_pred)

In [43]:
sub = pd.concat([test['id'],test_pred],axis=1)

In [44]:
sub.head(5)

,id,0
0,18009,british
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian


In [45]:
sub.to_csv('svm.csv', index=False)

###### Model 2: Random Forest

In [34]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(train_tfidf, y_train)
train_pred_rf = rf.predict(train_tfidf)
val_pred_rf = rf.predict(val_tfidf)
test_pred_rf = rf.predict(test_tfidf)

In [35]:
print(classification_report(y_train, train_pred_rf))
print(classification_report(y_test, val_pred_rf))

              precision    recall  f1-score   support

   brazilian       1.00      1.00      1.00       320
     british       1.00      1.00      1.00       583
cajun_creole       1.00      1.00      1.00      1059
     chinese       1.00      1.00      1.00      1846
    filipino       1.00      1.00      1.00       525
      french       1.00      1.00      1.00      1897
       greek       1.00      1.00      1.00       828
      indian       1.00      1.00      1.00      2096
       irish       1.00      1.00      1.00       476
     italian       1.00      1.00      1.00      5479
    jamaican       1.00      1.00      1.00       357
    japanese       1.00      1.00      1.00       968
      korean       1.00      1.00      1.00       574
     mexican       1.00      1.00      1.00      4543
    moroccan       1.00      1.00      1.00       578
     russian       1.00      1.00      1.00       337
 southern_us       1.00      1.00      1.00      3059
     spanish       1.00    

In [39]:
#from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import classification_report

#rfc = RandomForestClassifier(n_jobs=-1, max_features='sqrt') 
 
# Use a grid over parameters of interest
#param_grid = { 
           "n_estimators" : [300, 500, 700],
           "max_depth" : [None, 1, 2, 3],
           "min_samples_leaf" : [1, 2, 3]}

In [46]:
#clf = GridSearchCV(estimator=rfc, param_grid=param_grid,cv=5)
#clf.fit(train_tfidf, y_train)

In [ ]:
#clf.best_params_

In [ ]:
#print(clf.best_params_)

In [ ]:
#rfc_best = RandomForestClassifier(**clf.best_params_)
#rfc_best.fit(train_tfidf, y_train)
#train_pred_rfc_best = rfc_best.predict(train_tfidf)
#val_pred_rfc_best = rfc_best.predict(val_tfidf)
#test_pred_rfc_best = rfc_best.predict(test_tfidf)

In [45]:
print(classification_report(y_train, train_pred_rf))
print(classification_report(y_test, val_pred_rf))

              precision    recall  f1-score   support

   brazilian       1.00      1.00      1.00       320
     british       1.00      1.00      1.00       583
cajun_creole       1.00      1.00      1.00      1059
     chinese       1.00      1.00      1.00      1846
    filipino       1.00      1.00      1.00       525
      french       1.00      1.00      1.00      1897
       greek       1.00      1.00      1.00       828
      indian       1.00      1.00      1.00      2096
       irish       1.00      1.00      1.00       476
     italian       1.00      1.00      1.00      5479
    jamaican       1.00      1.00      1.00       357
    japanese       1.00      1.00      1.00       968
      korean       1.00      1.00      1.00       574
     mexican       1.00      1.00      1.00      4543
    moroccan       1.00      1.00      1.00       578
     russian       1.00      1.00      1.00       337
 southern_us       1.00      1.00      1.00      3059
     spanish       1.00    

In [47]:
test_predict_rf = pd.DataFrame(test_pred_rf)
sub = pd.concat([test['id'],test_predict_rf],axis=1)

In [48]:
sub.to_csv("rf.csv",index=True)

###### Model 3: Stochastic Gradient Descent 

In [41]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(shuffle=True, random_state=101)
sgd.fit(train_tfidf,y_train)
train_pred_sgd = sgd.predict(train_tfidf)
val_pred_sgd = sgd.predict(val_tfidf)
test_pred_sgd = sgd.predict(test_tfidf)

In [42]:
print(classification_report(y_train, train_pred_sgd))
print(classification_report(y_test, val_pred_sgd))

              precision    recall  f1-score   support

   brazilian       0.88      0.53      0.66       320
     british       0.90      0.44      0.59       583
cajun_creole       0.86      0.76      0.81      1059
     chinese       0.84      0.90      0.87      1846
    filipino       0.89      0.71      0.79       525
      french       0.77      0.64      0.70      1897
       greek       0.84      0.76      0.80       828
      indian       0.87      0.94      0.90      2096
       irish       0.86      0.50      0.63       476
     italian       0.77      0.95      0.85      5479
    jamaican       0.85      0.76      0.80       357
    japanese       0.93      0.72      0.81       968
      korean       0.92      0.80      0.86       574
     mexican       0.91      0.94      0.92      4543
    moroccan       0.91      0.76      0.83       578
     russian       0.87      0.52      0.65       337
 southern_us       0.74      0.87      0.80      3059
     spanish       0.88    

In [43]:
test_predict_sgd = pd.DataFrame(test_pred_sgd)
sub = pd.concat([test['id'],test_predict_sgd],axis=1)

In [44]:
sub.to_csv("sgd.csv",index=True)

Kaggle Dataset: Whats Cooking

Some of our strongest geographic and cultural associations are tied to a 
region's local foods. This playground competitions asks you to predict 
the category of a dish's cuisine given a list of its ingredients. 
Submissions are evaluated on the categorization accuracy 
(the percent of dishes that you correctly classify).Submission should 
predict the cuisine for each recipe in the test set. 

Data description: 
In the dataset, recipe id, the type of cuisine, and the list of ingredients 
of each recipe (of variable length) is included. The data is stored in 
JSON format. In the test file test.json, the format of a recipe is the same as train.json, only the cuisine type is removed, as
it is the target variable to be predicted.

Data Fields: 

train.json - the training set containing recipes id, type of cuisine, and list of ingredients
test.json - the test set containing recipes id, and list of ingredients
sample_submission.csv - a sample submission file in the correct format

Steps in Data Processing Conducted:
1. Removing space within words from the ingredients column 
2. Removing the list of lists within the 'ingredients' column
3. Creating an addditional column to remove the list of lists, hypen 
and then concatinating to orginal dataframe - now the new column 
is a cleaned column for applying TFIDF vectorisation 
4.  Data was split into 80:20 for training and validation 


Models Applied: 
Support Vector Machine, 
Random Forest, 
Stochastic Gradient Descent

Error Metrics for evaluation - Accuracy, recall, precision, F1 score.


Model Results: 

Support Vector Classifier performed better compared to Support Vector 
Machine, giving the overall ranking closer to 822 on the ranking on the 
Pvt Leaderscoreboard
